# Quantitative Trading Strategies Using Python
## Technical Analysis, Statistical Testing,and Machine Learning
—
### Peng Liu

#######################################################################################################################

## CHAPTER 1
## Quantitative Trading: An Introduction

### Getting Started with Financial Data Analysis

#### Downloading Stock Price Data

In [ ]:
!pip install botorch
!pip install -U statsmodels
!pip install ta
!pip install yfinance
from botorch.acquisition import ExpectedImprovement
from botorch.acquisition import qExpectedImprovement
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.knowledge_gradient import qKnowledgeGradient
from botorch.fit import fit_gpytorch_mll
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import combinations
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import requests
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import ta
import torch
import torch.nn as nn
import yfinance as yf

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


##### Use the Ticker module to access ticker data

In [ ]:
msft = yf.Ticker("MSFT")
# get stock info
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

##### Access a specific attribute, such as market capitilisation, from the dictionary

In [ ]:
msft.info["marketCap"]

3475050856448

##### Download daily stock price data by passing in specified ticker and date range

In [ ]:
today_date = datetime.today().strftime('%Y-%m-%d')
print(today_date)
data = yf.download("MSFT", start="2022-01-01", end=today_date)

[*********************100%%**********************]  1 of 1 completed

2024-07-06


In [ ]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-28,453.070007,455.380005,446.410004,446.950012,446.950012,28362300
2024-07-01,448.660004,457.369995,445.660004,456.730011,456.730011,17662800
2024-07-02,453.200012,459.589996,453.109985,459.279999,459.279999,13979800
2024-07-03,458.190002,461.019989,457.880005,460.769989,460.769989,9932800
2024-07-05,459.609985,468.350006,458.970001,467.559998,467.559998,15980200


##### Check data dimension/size

In [ ]:
data.shape

(629, 6)

#### Visualizing Stock Price Data

##### Plot closing price as a time series chart

In [ ]:
fig = go.Figure(data=go.Scatter(x=data.index,y=data['Close'],mode='lines'))
fig.show()

##### Overlay the trading volume

In [ ]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(go.Scatter(x=data.index,y=data['Close'],name='Price'),secondary_y=False)
fig2.add_trace(go.Bar(x=data.index,y=data['Volume'],name='Volume'),secondary_y=True)
fig2.show()

##### Rescale volume

In [ ]:
fig2.update_yaxes(range=[0,500000000],secondary_y=True)
fig2.update_yaxes(visible=True, secondary_y=True)
fig2

##### Switch to candlestick chart

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Candlestick(x=data.index,
open=data['Open'],
high=data['High'],
low=data['Low'],
close=data['Close'],
))
fig3

##### Overlay the trading volume and rescale volume

In [ ]:
fig4 = make_subplots(specs=[[{"secondary_y": True}]])
fig4.add_trace(go.Scatter(x=apple.index,y=data['Close'],name='Price'),secondary_y=False)
fig4.add_trace(go.Bar(x=apple.index,y=data['Volume'],name='Volume'),secondary_y=True)

fig4.update_yaxes(range=[0,200000000],secondary_y=True)
fig4.update_yaxes(visible=True, secondary_y=True)
fig4

##### Switch to candlestick chart

In [ ]:
fig5 = make_subplots(specs=[[{"secondary_y": True}]])
fig5.add_trace(go.Candlestick(x=data.index,
open=data['Open'],
high=data['High'],
low=data['Low'],
close=data['Close'],
))
fig5

##### Get year-to-date (YTD) average stock price

In [ ]:
apple = yf.download("AAPL", start="2022-01-01", end=today_date)
end_year_apple = apple.groupby(pd.Grouper(level='Date', freq='YE')).nth(-1).reset_index()
start_year_apple= apple.groupby(pd.Grouper(level='Date', freq='YE')).nth(0).reset_index()

ytd = ((end_year_apple['Close'] - start_year_apple['Close']) / start_year_apple['Close']) *100

[*********************100%%**********************]  1 of 1 completed


In [ ]:
ytd

0   -28.613814
1    53.937794
2    21.924153
Name: Close, dtype: float64

#######################################################################################################################